<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 15px">
# Logistic Regression, kNN, and ROC/AUC

---

This is an open-ended lab.

You will:

1. Load in the wine dataset (create target, concatenate, normalize predictors).
- Do EDA on predictors.
- Select predictors of interest.
- Load KNeighborsClassifier and LogisticRegression from sklearn.
- Compare performance between the two using stratified cross-validation.
- Plot and evaluate the ROC curve for your classifiers.

---

### Humor styles questions encoding reference

### 32 questions:

Subjects answered **32** different questions outlined below:

1. I usually don't laugh or joke with other people.
2. If I feel depressed, I can cheer myself up with humor.
3. If someone makes a mistake, I will tease them about it.
4. I let people laugh at me or make fun of me at my expense more than I should.
5. I don't have to work very hard to make other people laugh. I am a naturally humorous person.
6. Even when I'm alone, I am often amused by the absurdities of life.
7. People are never offended or hurt by my sense of humor.
8. I will often get carried away in putting myself down if it makes family or friends laugh.
9. I rarely make other people laugh by telling funny stories about myself.
10. If I am feeling upset or unhappy I usually try to think of something funny about the situation to make myself feel better.
11. When telling jokes or saying funny things, I am usually not concerned about how other people are taking it.
12. I often try to make people like or accept me more by saying something funny about my own weaknesses, blunders, or faults.
13. I laugh and joke a lot with my closest friends.
14. My humorous outlook on life keeps me from getting overly upset or depressed about things.
15. I do not like it when people use humor as a way of criticizing or putting someone down.
16. I don't often say funny things to put myself down.
17. I usually don't like to tell jokes or amuse people.
18. If I'm by myself and I'm feeling unhappy, I make an effort to think of something funny to cheer myself up.
19. Sometimes I think of something that is so funny that I can't stop myself from saying it, even if it is not appropriate for the situation.
20. I often go overboard in putting myself down when I am making jokes or trying to be funny.
21. I enjoy making people laugh.
22. If I am feeling sad or upset, I usually lose my sense of humor.
23. I never participate in laughing at others even if all my friends are doing it.
24. When I am with friends or family, I often seem to be the one that other people make fun of or joke about.
25. I donít often joke around with my friends.
26. It is my experience that thinking about some amusing aspect of a situation is often a very effective way of coping with problems.
27. If I don't like someone, I often use humor or teasing to put them down.
28. If I am having problems or feeling unhappy, I often cover it up by joking around, so that even my closest friends don't know how I really feel.
29. I usually can't think of witty things to say when I'm with other people.
30. I don't need to be with other people to feel amused. I can usually find things to laugh about even when I'm by myself.
31. Even if something is really funny to me, I will not laugh or joke about it if someone will be offended.
32. Letting others laugh at me is my way of keeping my friends and family in good spirits.

---

### Response scale:

For each question, there are 5 possible response codes ("likert scale") that correspond to different answers. There is also a code that indicates there is no response for that subject.

    1 == "Never or very rarely true"
    2 == "Rarely true"
    3 == "Sometimes true"
    4 == "Often true"
    5 == "Very often or always true
    [-1 == Did not select an answer]
    
---

### Demographics:

    age: entered as as text then parsed to an interger.
    gender: chosen from drop down list (1=male, 2=female, 3=other, 0=declined)
    accuracy: How accurate they thought their answers were on a scale from 0 to 100, answers were entered as text and parsed to an integer. They were instructed to enter a 0 if they did not want to be included in research.	

---

### 1. Load humor styles dataset

In [128]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#What is the SKLearn Pipeline? and how can I use it?

In [65]:
hsq = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/datasets/humor_styles/hsq_data.csv')

In [66]:
hsq.shape

(1071, 39)

In [67]:
hsq.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q30,Q31,Q32,affiliative,selfenhancing,agressive,selfdefeating,age,gender,accuracy
0,2,2,3,1,4,5,4,3,4,3,...,4,2,2,4.0,3.5,3.0,2.3,25,2,100
1,2,3,2,2,4,4,4,3,4,3,...,4,3,1,3.3,3.5,3.3,2.4,44,2,90
2,3,4,3,3,4,4,3,1,2,4,...,5,4,2,3.9,3.9,3.1,2.3,50,1,75
3,3,3,3,4,3,5,4,3,-1,4,...,5,3,3,3.6,4.0,2.9,3.3,30,2,85
4,1,4,2,2,3,5,4,1,4,4,...,5,4,2,4.1,4.1,2.9,2.0,52,1,80


In [109]:
hsq = hsq[(hsq['age'] <= 100)]
hsq = hsq[hsq['affiliative'] <= 5.0]
#hsq['gender'] = hsq['gender'].map(lambda x: 1 if 1)

In [131]:
hsq.corr()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q30,Q31,Q32,affiliative,selfenhancing,agressive,selfdefeating,age,gender,accuracy
Q1,1.000000,-0.182828,-0.150152,-0.084227,-0.393875,-0.234358,-0.036678,-0.110455,0.308965,-0.192480,...,-0.050889,0.143377,-0.161843,-0.707386,-0.257959,-0.008108,-0.143732,-0.011980,0.012171,-0.124674
Q2,-0.182828,1.000000,0.115359,0.105082,0.215852,0.285450,0.008464,0.113116,-0.162391,0.449402,...,0.255529,-0.039368,0.149628,0.294955,0.644425,0.067491,0.158724,0.006319,-0.017527,0.137898
Q3,-0.150152,0.115359,1.000000,0.180742,0.179603,0.143360,-0.258582,0.167085,-0.131985,0.122729,...,0.056335,-0.310602,0.212763,0.225939,0.162452,0.214457,0.251455,-0.220749,-0.122043,0.027224
Q4,-0.084227,0.105082,0.180742,1.000000,0.090255,0.108772,-0.052856,0.449436,-0.107374,0.150071,...,0.032838,-0.049467,0.453536,0.086909,0.134807,0.064842,0.690992,-0.082052,-0.089823,-0.002573
Q5,-0.393875,0.215852,0.179603,0.090255,1.000000,0.344271,0.044010,0.155272,-0.317519,0.223618,...,0.136585,-0.064759,0.170603,0.690975,0.308401,0.177195,0.186857,0.053595,-0.090095,0.173414
Q6,-0.234358,0.285450,0.143360,0.108772,0.344271,1.000000,-0.026224,0.111740,-0.176641,0.307462,...,0.419304,-0.053729,0.087285,0.368612,0.596897,0.097326,0.145287,0.003628,-0.010651,0.138154
Q7,-0.036678,0.008464,-0.258582,-0.052856,0.044010,-0.026224,1.000000,-0.039595,0.018590,-0.002645,...,0.021725,0.387676,-0.006572,0.023054,0.025356,0.294052,-0.076626,0.108588,0.103991,0.049568
Q8,-0.110455,0.113116,0.167085,0.449436,0.155272,0.111740,-0.039595,1.000000,-0.130868,0.186186,...,0.022428,-0.011314,0.488592,0.143384,0.167563,0.120648,0.726642,-0.003863,-0.058775,-0.022615
Q9,0.308965,-0.162391,-0.131985,-0.107374,-0.317519,-0.176641,0.018590,-0.130868,1.000000,-0.168571,...,-0.064266,0.108410,-0.156256,-0.602986,-0.190005,0.032642,-0.150497,-0.048346,-0.006854,-0.062504
Q10,-0.192480,0.449402,0.122729,0.150071,0.223618,0.307462,-0.002645,0.186186,-0.168571,1.000000,...,0.264062,-0.025921,0.187974,0.259388,0.743120,0.114708,0.256834,0.052520,0.014044,0.087315


---

### 2. Create a target and predictor matrix

Target and predictors are up to you. 

In [110]:
X = hsq[['affiliative','selfenhancing','agressive','selfdefeating']]
y = hsq['gender']

print X.shape, y.shape

(1066, 4) (1066,)


---

### 3. Perform any EDA you deem relevant on your predictors and target

In [ ]:
#DONE

---

### 4. Perform stratified cross-validation on a KNN classifier and logisitic regression.

1. Gridsearch the best KNN parameters.

Note: cross_val_score conveniently does stratification for you when you have a categorical target. :/ So much for forcing you to practice StratifiedKFold...

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, stratify=y)

In [118]:
knn = KNeighborsClassifier()
knn_params = {'n_neighbors' : range(1, 101, 2), 'weights': ['distance', 'uniform']}
knn_gs1 = GridSearchCV(knn, knn_params, cv=10, verbose=1, n_jobs=-1)
knn_gs2 = GridSearchCV(knn, knn_params, cv=10, verbose=1, n_jobs=-1)
result = knn_gs1.fit(X_train, y_train)
result = knn_gs2.fit(X, y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done 628 tasks      | elapsed:    5.1s


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.3s finished
[Parallel(n_jobs=-1)]: Done 628 tasks      | elapsed:    5.4s


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.8s finished


In [125]:
print 'Best Score:', knn_gs1.best_score_
print 'Best Score:', knn_gs2.best_score_
print 'Best Params:', knn_gs1.best_params_
print 'Best Params:', knn_gs2.best_params_

Best Score: 0.571964956195
Best Score: 0.55722326454
Best Params: {'n_neighbors': 9, 'weights': 'distance'}
Best Params: {'n_neighbors': 13, 'weights': 'uniform'}


---

### 5. Regularization with logistic regression

Logistic regression can also use the Lasso and Ridge penalties.

The `penalty` keyword argument can be set to `l2` for Ridge and `l1` for Lasso. 

Note: you must set `solver='liblinear'` if you're going to use the Lasso penalty!

**`C` is the regularization strength for LogisticRegression, but IT IS THE INVERSE OF ALPHA: 1/alpha. I don't know why they did this – it's stupid.**

1. Select everything but your target to be predictors.
- Normalize the predictors!
- Gridsearch the LogisticRegression with regularization.
- Gridsearch the KNN.
- Compare their cross-validated accuracies.

---

### 6. Explain why that regularization for logistic regression may have been chosen. Print out the most important variables for predicting your target from logistic regression.

---

### 7. Re-run a (non-regularized) logistic regression with only centered coefficients (not normalized). Interperet the baseline probability and the effect of one of your predictors.

**sklearn's LogisticRegression actually uses l2 Ridge regularization by default with `C=1`! To "turn it off" set `C=1e10`.**

1. Fit the logistic regression using centered predictors.
- Write a function to turn coefficient results to probability (logistic transformation).
- Describe the baseline probability.
- Plot the distribution of one of your predictors.
- Describe based on the coefficient of the predictor the effect on probability of your target variable.

---

### 8. Plot the ROC curve for one or more of your models

Some code is provided for you.

What is the intuition for the ROC curve? As the class assignment threshold increases for the positive class, the false positive rate and true positive rate necessarily increase. For a classifier performing at chance, this would be the diagonal dotted line: an equal chance of false positives and true positives.

- The greater the area under the curve, the better higher the quality of the model
- The greater the area under the curve, the higher the ratio of true positives to false positives as the threshold becomes more lenient. Thus, the greater the area under the curve, the higher the quality of the classification model.


In [3]:
# from sklearn.metrics import roc_curve, auc

# # Get out the predicted probabilities for the X observation matrix
# y_pp = lr_centered.predict_proba(Xc)

# # roc_curve returns the false positive rate and true positive rates as the threshold changes
# # takes in the y and the predicted probabilities of the positive class from your model.
# fpr, tpr, _ = roc_curve(y, y_pp[:, 1])
# roc_auc = auc(fpr, tpr)

# plt.figure(figsize=[9,9])
# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc, linewidth=4)
# plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate', fontsize=16)
# plt.ylabel('True Positive Rate', fontsize=16)
# plt.title('Receiver operating characteristic curve', fontsize=20)
# plt.legend(loc="lower right")
# plt.show()